# Projeto 01: Reconhecimento de Dígitos

### João Victor Soares Silva | Mat: 20210027300
### Yhasmim de Souza Tigre | Mat: 20210026966 

## Pré-Processamento

In [13]:
import numpy as np
import pandas as pd

In [23]:
N = 28 #Número de linhas e colunas das matrizes das imagens
P_MAX = 255 #Número máximo de um pixel

In [21]:
'''
Aqui é só um exemplo menor de como funcionam as fórmulas da simetria no array.
Acho bom usar pra estudar pra apresentação, então deixei aqui.
'''

teste = [
 1, 2, 3, 4,
 5, 6, 7, 8,
 9, 10, 11, 12,
 13, 14, 15, 16]

lin = 4

#Vertical
print('Vertical')
for i in range(lin):

    for j in range(int(lin/2)):
        print(f'{teste[(lin*i) + j]} + {teste[(lin*i) + (lin - j - 1)]} =
                {teste[(lin*i) + j] + teste[(lin*i) + (lin - j - 1)]}')

#Horizontal
print('\nHorizontal')
for i in range(int(lin/2)):
    
    for j in range(lin):
        print(f'{teste[(lin*i) + j]} + {teste[(lin*(lin - i - 1)) + j]} =
                {teste[(lin*i) + j] + teste[(lin*(lin - i - 1)) + j]}')

'\nAqui é só um exemplo menor de como funcionam as fórmulas da simetria no array.\nAcho bom usar pra estudar pra apresentação, então deixei aqui.\n'

In [24]:
def reduzir (nome : str, arq : str) -> None:
    aux = np.loadtxt (arq, delimiter = ';', dtype = str)
    aux = np.delete (aux, 0, 0) #Removendo a linha dos nomes das colunas
    lista_y = []
    lista_intensidade = []
    lista_simetria_vertical = []
    lista_simetria_horizontal = []
    lista_simetria = []

    for i in range (len(aux)): #Salvando as labels
        lista_y.append (int(aux[i][0]))
    
    aux = np.delete (aux, 0, 1) #Removendo as labels
    
    for i in range (len(aux)): #Calculando as intensidades
        soma = 0    

        for j in range (len(aux[0])):
            soma += int(aux[i][j])
        
        lista_intensidade.append(soma/P_MAX)
    
    for imagem in aux: #Simetria vertical
        soma = 0

        for i in range(N):
            for j in range(int(N/2)):
                soma += abs(int(imagem[(N*i) + j]) - int(imagem[(N*i) + (N - j - 1)]))

        lista_simetria_vertical.append(soma/P_MAX)

    for imagem in aux: #Simetria horizontal
        soma = 0

        for i in range(int(N/2)):
            for j in range(N):
                soma += abs(int(imagem[(N*i) + j]) - int(imagem[N * (N - i - 1) + j]))

        lista_simetria_horizontal.append(soma/P_MAX)
    
    for i in range (len(aux)): #Somando as simetrias
        lista_simetria.append(lista_simetria_vertical[i] + lista_simetria_horizontal[i])

    #Criando e preenchendo o novo .csv
    dados_redux = open (f'{nome}.csv', 'w')

    dados_redux.write ('label;intensidade;simetria\n')
    for i in range (len(aux)):
        dados_redux.write (f'{lista_y[i]};{lista_intensidade[i]};{lista_simetria[i]}\n')
    
    dados_redux.close()

    return

In [25]:
reduzir(nome = 'train_redu', arq = 'train.csv')
reduzir(nome = 'test_redu', arq = 'test.csv')

In [26]:
def eval(nome_train: str, nome_eval: str):
    # Carrega os dados de treino
    train_data = np.loadtxt('train_redu.csv', delimiter=';', dtype=str)
    
    # Ignora a primeira linha
    train_data = np.delete(train_data, 0, 0)
    
    # Calcula o tamanho dos conjuntos de treino e avaliação
    train_len = int(0.8 * len(train_data))
    eval_len = len(train_data) - train_len
    
    # Escreve os cabeçalhos nos arquivos
    with open(f'{nome_train}.csv', 'w') as train_file:
        train_file.write('label;intensidade;simetria\n')
    
    with open(f'{nome_eval}.csv', 'w') as eval_file:
        eval_file.write('label;intensidade;simetria\n')
    
    # Divide os dados em conjuntos de treino e avaliação
    np.savetxt(f'{nome_train}.csv', train_data[:train_len], delimiter=';', fmt='%s')
    np.savetxt(f'{nome_eval}.csv', train_data[train_len:], delimiter=';', fmt='%s')


    #não retorna corretamente - precisa retornar amostra aleatória de treino e avaliação

In [27]:
eval('train_redu', 'eval_redu')

ValueError: a must be 1-dimensional

## Algoritmos

### Perceptron Learning Algorithm (PLA)

In [24]:
class PLA ():
    def __init__(self, arq : str, w_lista = []) -> None:
        '''
        Entradas : arq - .csv dos dados, contendo suas características e rotulação. 
        '''
        self.xy_lista = []
        self.w_lista = w_lista

        aux = np.loadtxt (arq, delimiter = ';', dtype = str)
        aux = np.delete (aux, 0, 0) #Removendo a linha dos nomes das colunas

        for linha in aux:
            self.xy_lista.append([[1, linha[1], linha[2]], linha[0]]) #O 1 será usado para a multiplicação com o bias posteriormente
        
        if (len(w_lista) == 0):
            for i in range (3): #Já que temos 2 features + o bias
                self.w_lista.append(0) #Os pesos inicializarão em 0, caso não existam

    def Acuracia (self, w_lista : list) -> float:
        soma_PCC = 0

        for i in range (len(self.xy_lista)):
            aux = np.sign (np.sum(np.multiply(w_lista, self.xy_lista[i][0])))

            if (aux == self.xy_lista[i][1]):
                soma_PCC += 1
    
        return (soma_PCC/len(self.xy_lista))

    def ConstrutorLista_PCI (self, pontos : list) -> list:
        lista_PCI = []

        for i in range (len(pontos)):
            aux = np.sign (np.sum(np.multiply(self.w_lista, pontos[i][0])))

            if (aux != pontos[i][1]):
                lista_PCI.append(pontos[i])
        
        return lista_PCI

    def ExecutarPLA (self, n_int = 100) -> None:
        lista_PCI = self.xy_lista
        w_otimo = self.w_lista

        i = 0
        while (len(lista_PCI) > 0 or i < n_int):
            ale_index = np.random.randint(0, len(lista_PCI)) #index aleatório
            ponto = lista_PCI[ale_index]

            aux = np.multiply(ponto[0], ponto[1]) #x * y
            w_novo = np.add(self.w_lista, aux)

            if (self.Acuracia(w_lista = w_otimo) < self.Acuracia(w_lista = w_novo)):
                w_otimo = w_novo

            self.w_lista = w_novo

            lista_PCI = self.ConstrutorLista_PCI (self, pontos = self.xy_lista)
            i += 1
        
        self.w_lista = w_otimo

        return


### Regressão Linear

In [ ]:
class LinearRegression:
    def fit(self, _X, _y):
      #criando matrizes
      x = np.array(_X)
      y = np.array(_y)

      #vetor de pesos w (seguindo o calculo matematico)
      self.w = np.linalg.inv(x.transpose() * x) * x.transpose() * y

      return self.w

    def predict(self, _x):#lista de amostras
      #retorna uma lista de classificações

      return [np.dot(_x, self.w) for x in _x]

    def getW(self):
        return self.w

In [ ]:
"""
Teste RLinear para ser adaptado
_timeRL = [[1, t] for t in _time]

lr = LinearRegression()
lr.fit(_timeRL, _data)

_dataRL = lr.predict(_timeRL)

_timeLabelForecast = time[96:]
_timeForecast = [i for i in range(len(_timeLabel), len(_timeLabel) + len(_timeLabelForecast))]
_dataForecast = lr.predict([[1, t] for t in _timeForecast])




config_plt()

plt.plot(_timeLabel, _data, 'r-*')
plt.plot(_timeLabel, _dataRL, 'k--')
plt.plot(_timeLabelForecast, _dataForecast, 'b*')

plt.legend(['~f(.)', 'h(.)', 'previsão'], loc=2)"""

### Regressão Logística

In [25]:
class Reg_Log ():
    def __init__ (self) -> None:
        pass

## Classificação dos Dígitos 1x5

In [ ]:
#remover linhas com determinado valor
def class1x5 (nome : str):
    
    df = open (f'{nome}.csv', 'r')

#plotar dados de train 1x5 em um graf de 2 dimensões
# label = 1 azul, labl = 5, vermelho  


In [20]:
# Carregar os arquivos CSV
train_data = pd.read_csv('train_redu.csv')
test_data = pd.read_csv('test_redu.csv')

# Filtrar os dados para deixar apenas as imagens com valores 1 ou 5 na coluna label
train1x5 = train_data[train_data['label'].isin([1, 5])]
test1x5 = test_data[test_data['label'].isin([1, 5])]

# Plotar os dados de train1x5 em um gráfico de duas dimensões (intensidade X simetria)
plt.figure(figsize=(8, 6))
plt.scatter(train1x5['intensity'], train1x5['symmetry'], c=train1x5['label'], cmap='coolwarm', alpha=0.6)
plt.xlabel('Intensidade')
plt.ylabel('Simetria')
plt.title('Gráfico de Dispersão - Intensidade vs Simetria (Train1x5)')
plt.colorbar(label='Label')
plt.grid(True)
plt.show()

KeyError: 'label'

#X = intensidade, simetria  para toda imagem de train 1x5

#atribuir valor alvo y = +1 para label = 1 

#atribuir valor alvo y = -1 para label = 5


### Perceptron Learning Algorithm (PLA)

In [ ]:
#treino com train1x5

### Regressão Linear

In [ ]:
#treino com train1x5

### Regressão Logística

In [ ]:
#treino com train1x5

### Relatório de Eficácia

In [ ]:
#para cada classificador

from sklearn.metrics import classification_report
form sklearn.metrics import confusion_matrix
#print(classification_report(y_true, y_pred, target_names=target_names)) EXEMPLO
#print(confusion_matrix(y_true, y_pred, *, labels=None, sample_weight=None, normalize=None)) EXEMPLO

## Classificação de Dígitos Completo

### Perceptron Learning Algorithm (PLA)

### Regressão Linear

### Regressão Logística

### Relatório de Eficácia

In [ ]:
#print(classification_report(y_true, y_pred, target_names=target_names)) EXEMPLO

## Comparação dos Classificadores

## Implementações Avançadas

### Weight-Decay para a Regressão Logística

In [ ]:
#os pesos da função w serão penalisados
#o w que possui o melhor eval é o escolhido

### Definindo a Melhor Ordem